<a href="https://colab.research.google.com/github/AaravDB/ML/blob/testing/NaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task-0

In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [3]:
def dataset_splitter(filename,label_column_name):
  dat=pd.read_csv(filename)
  y=np.array(dat[label_column_name])
  tempdat=dat.drop(label_column_name,axis=1)
  X=np.array(tempdat)
  return train_test_split(X,y,train_size=12/14,random_state=42)
X_t,X_te,y_t,y_te=dataset_splitter('Data.csv','Play')

In [4]:
print(y_t)

['no' 'yes' 'no' 'yes' 'yes' 'no' 'no' 'yes' 'no' 'yes' 'yes' 'yes']


In [5]:
print(X_t)

[['Rainy' 'Hot' 'High' 'f']
 ['Overcast' 'Hot' 'Normal' 'f']
 ['Sunny' 'Cool' 'Normal' 't']
 ['Rainy' 'Cool' 'Normal' 'f']
 ['Overcast' 'Hot' 'High' 'f']
 ['Rainy' 'Hot' 'High' 't']
 ['Sunny' 'Mild' 'High' 't']
 ['Sunny' 'Cool' 'Normal' 'f']
 ['Rainy' 'Mild' 'High' 'f']
 ['Rainy' 'Mild' 'Normal' 't']
 ['Sunny' 'Mild' 'High' 'f']
 ['Overcast' 'Cool' 'Normal' 't']]


Task-1

In [6]:
def prior_calculator(labels):
  prob=np.zeros(len(np.unique(labels)))
  for labl in labels:
    if labl=='no':
      prob[0]+=1
    else:
      prob[1]+=1
  prob=prob/np.sum(prob)
  return prob

In [7]:
prior=prior_calculator(y_t)
print(prior)

[0.41666667 0.58333333]


Task-2

In [8]:
def likelihood_calculator(data,labels):
  #in the end of the likelihood calculation we need p(x1,x2...xn|y)=p(x1|y)*p(x2|y)*...*p(xn|y) for each label thus we input label
  #p(x1|y)=[p(x11|y),p(x12|y)...]
  m,n=data.shape
  unique_labels=np.unique(labels)
  likelihoods={}
  for label in unique_labels:
    label_likelihoods={}
    for i in range(n):
        #iterating over all features
      feat_vals=data[:,i]
      unique_feat_vals=np.unique(feat_vals)
      feat_likelihood={}
      for unique_feat_value in unique_feat_vals:
        label_ct=0
        ct_unique_value=0
        for j in range(m):
          #iterating over each sample's feature value
          if  labels[j]==label:
            label_ct+=1
            if feat_vals[j]==unique_feat_value:
              ct_unique_value+=1
        if label_ct!=0:
          feat_likelihood[unique_feat_value]=ct_unique_value/label_ct
        else:
          feat_likelihood[unique_feat_value]=0
      label_likelihoods[i]=feat_likelihood
    likelihoods[label]=label_likelihoods
  return likelihoods

likelihoods=likelihood_calculator(X_t,y_t)
print(likelihoods)

{'no': {0: {'Overcast': 0.0, 'Rainy': 0.6, 'Sunny': 0.4}, 1: {'Cool': 0.2, 'Hot': 0.4, 'Mild': 0.4}, 2: {'High': 0.8, 'Normal': 0.2}, 3: {'f': 0.4, 't': 0.6}}, 'yes': {0: {'Overcast': 0.42857142857142855, 'Rainy': 0.2857142857142857, 'Sunny': 0.2857142857142857}, 1: {'Cool': 0.42857142857142855, 'Hot': 0.2857142857142857, 'Mild': 0.2857142857142857}, 2: {'High': 0.2857142857142857, 'Normal': 0.7142857142857143}, 3: {'f': 0.7142857142857143, 't': 0.2857142857142857}}}


In [9]:

def predict(samples,train_labels):
  unique_labels=np.unique(train_labels)
  likelihoods=likelihood_calculator(samples,train_labels)
  m,n=samples.shape
  predictions=[]
  for sample in samples:
    best_label=train_labels[m-1]#anything
    max_post=-15999
    prir=prior_calculator(train_labels)
    for i in range(len(unique_labels)):
      curr_post=np.log(prir[i])
      label=unique_labels[i]
      for j in range(n):
        valu=likelihoods[label][j][sample[j]]
        if valu!=0:
          curr_post+=np.log(valu)
        else:
          curr_post+=-100
      if curr_post>max_post:
        max_post=curr_post
        best_label=label
    predictions.append(best_label)
  predictions=np.array(predictions)
  return predictions


In [10]:
test_predictions=predict(X_te,y_t)
train_predictions=predict(X_t,y_t)

In [11]:
def accuracy_finder(true_labls,predicted_labls):
  acc_ct=0
  for i in range(len(true_labls)):
    if predicted_labls[i]==true_labls[i]:
      acc_ct+=1
  return acc_ct/len(true_labls)


In [12]:
train_acc=accuracy_finder(y_t,train_predictions)
test_acc=accuracy_finder(y_te,test_predictions)

In [13]:
print(train_acc)
print(test_acc)

0.75
0.5
